In [ ]:
# !pip install emot
# !pip install symspellpy
# !pip install setuptools wheel
# !pip install spacy
# !pip install pyspellchecker
# !pip install spellchecker
# !pip install inflect
# !pip install langdetect
# !pip install google_trans_new
# !pip install deep-translator
# !pip install ekphrasis
# !pip install cdifflib
# !pip install gensim
# !pip install wordninja

In [2]:
import nltk
from nltk.corpus import PlaintextCorpusReader
import re, string, json

from symspellpy.symspellpy import SymSpell, Verbosity
import pkg_resources
import spacy
from tqdm.notebook import tqdm
from spellchecker import SpellChecker
import inflect

import difflib
import gensim

import pandas as pd

In [2]:
df = pd.read_csv("../data/usa_combined.csv")
df

,Text
0,want find way end racism try force people inje...
1,"this child almost died tetanus, spent months h..."
2,juste se rappeler m. leblanc la pression que l...
3,"you would expect cases rise significantly, cou..."
4,depends whether vaccination status is embedded...
...,...
423590,do it others.... 💪🎉💉🤙 #vaccinesaveslives
423591,got my first pfizer shot today. feeling well g...
423592,#fauciouchie #2 #vaccinated #vaccinesaveslives...
423593,yay 🙌🏻 congrats dr. j being fully vaccinated i...


In [3]:
df.drop_duplicates(keep='first', inplace=True)

In [4]:
df

,Text
0,want find way end racism try force people inje...
1,"this child almost died tetanus, spent months h..."
2,juste se rappeler m. leblanc la pression que l...
3,"you would expect cases rise significantly, cou..."
4,depends whether vaccination status is embedded...
...,...
423588,"if you are eligible, please pre- register covi..."
423590,do it others.... 💪🎉💉🤙 #vaccinesaveslives
423592,#fauciouchie #2 #vaccinated #vaccinesaveslives...
423593,yay 🙌🏻 congrats dr. j being fully vaccinated i...


## Used hashtags

In [5]:
text = df['Text']
hashtag_list = []
for tweet in text:
#     print(tweet)
    hashtags = re.findall("#[A-Za-z0-9_]+", tweet)
    if len(hashtags) > 0:
        for tag in hashtags:
            hashtag_list.append(tag)

In [6]:
hashtag_list

['#covidvaccineconfidence',
 '#vanguardvaccinationproject',
 '#populationcontrol',
 '#omicron',
 '#vaccination',
 '#mentalhealth',
 '#psychtwitter',
 '#cdnpoli',
 '#onpoli',
 '#covid19',
 '#l',
 '#polqc',
 '#getvaccinated',
 '#vaccineswork',
 '#canada',
 '#superbowlmusicfest',
 '#socal',
 '#superbowlexperience',
 '#sblvi',
 '#hiringphenomenaltalent',
 '#security',
 '#covid19vaccine',
 '#vhcon',
 '#covid19vaccine',
 '#vhcon',
 '#covid19vaccine',
 '#vhcon',
 '#legaultmustresign',
 '#legault',
 '#covid19',
 '#localnews',
 '#newsbreak',
 '#covid',
 '#defundcbc',
 '#defundbbc',
 '#nyc',
 '#godblessamerica',
 '#nonads',
 '#votehelenkicknadlerny10',
 '#dyk',
 '#fightflu',
 '#heartmonth',
 '#nowplaying',
 '#nowspinning',
 '#np',
 '#nowstreaming',
 '#subgenius',
 '#joerogan',
 '#antivaxxers',
 '#covidab',
 '#covid19',
 '#repnotrip',
 '#ableg',
 '#ciuss_mcq',
 '#covid19',
 '#ambulance',
 '#inacceptable',
 '#k',
 '#jdq',
 '#covid19',
 '#mybodymychoice',
 '#rosiebartonappreciationday',
 '#ohio',
 

## Emoji Processing

In [7]:
import pickle
from emot.emo_unicode import UNICODE_EMOJI, EMOTICONS_EMO

In [8]:
# # 'Emoji_Dict.p'- download link https://drive.google.com/open?id=1G1vIkkbqPBYPKHcQ8qy0G2zkoab2Qv4v
with open('./Emoji_Dict.p', 'rb') as fp:
    Emoji_Dict = pickle.load(fp)
Emoji_Dict = {v: k for k, v in Emoji_Dict.items()}

# # Function for converting emojis into word
def convert_emojis(text):
    for emot in UNICODE_EMOJI:
        text = text.replace(emot, "_".join(UNICODE_EMOJI[emot].replace(",","").replace(":","").split()))
    return text

In [9]:
# TEST: line of text before emoji processing
df["Text"][107]

'amoa resort has activated its covid-19 protocols will welcome guests as it strictly observes social distancing, face masks wearing enforce vaccination cards-only entry.'

In [10]:
# TEST: convert_emoji function
convert_emojis(df['Text'][107])

'amoa resort has activated its covid-19 protocols will welcome guests as it strictly observes social distancing, face masks wearing enforce vaccination cards-only entry.'

In [ ]:
# emoji processing on df["text"] column

df["Text"] = df["Text"].astype(str)
df["new_text"] = df["Text"].apply(convert_emojis)

In [ ]:
df["new_text"][107]

## Emoticon Processing

In [11]:
# 'Emoticon_Dict.p'- download link https://drive.google.com/open?id=1HDpafp97gCl9xZTQWMgP2kKK_NuhENlE
with open('./Emoticon_Dict.p', 'rb') as fp:
    Emoticon_Dict = pickle.load(fp)
    
# Function for converting emoticons into word
def convert_emoticons(text):
    for emot in EMOTICONS_EMO:
        text = text.replace(emot, "_".join(EMOTICONS_EMO[emot].replace(",","").replace(":","").split()))
    return text

In [47]:
# TEST: convert_emoticons function
sentence_listtext = "Hello :-) :-)"
convert_emoticons(sentence_listtext)

'Hello Happy_face_smiley Happy_face_smiley'

In [12]:
df["new_text"] = df["new_text"].apply(convert_emoticons)

In [13]:
df.head()

,Text,new_text
0,want find way end racism try force people inje...,want find way end racism try force people inje...
1,"this child almost died tetanus, spent months h...","this child almost died tetanus, spent months h..."
2,juste se rappeler m. leblanc la pression que l...,juste se rappeler m. leblanc la pression que l...
3,"you would expect cases rise significantly, cou...","you would expect cases rise significantly, cou..."
4,depends whether vaccination status is embedded...,depends whether vaccination status is embedded...


In [14]:
# to deal with utf-8 that pycld can't handle
import regex

RE_BAD_CHARS = regex.compile(r"\p{Cc}|\p{Cs}")

def remove_bad_chars(text):
    return RE_BAD_CHARS.sub("", text)

remove_bad_chars("A\x96 bad char")  # Cc category
# 'A bad char'

'A bad char'

In [15]:
df['new_text'] = df['new_text'].apply(remove_bad_chars)

## Detecting Languages

In [16]:
import pycld2 as cld2

tweets = df['new_text']
language = []

for each in tweets:
    text_content = each
    _, _, _, detected_language = cld2.detect(text_content, returnVectors=True)
    language.append(detected_language)

In [17]:
langs = []

for each in language:
    lang_detected = ""
    for i in range(0,len(each)):
        lang_extracted = each[i][2]
        if i != (len(each) - 1):
            lang_detected += lang_extracted + ", "
        else:
            lang_detected += lang_extracted
    langs.append(lang_detected)

In [18]:
langs

['ENGLISH',
 'ENGLISH',
 'FRENCH',
 'ENGLISH',
 'ENGLISH',
 'ENGLISH',
 'ENGLISH',
 'ENGLISH',
 'ENGLISH',
 'ENGLISH',
 'ENGLISH',
 'ENGLISH',
 'ENGLISH',
 'Unknown',
 'ENGLISH',
 'FRENCH',
 'ENGLISH',
 'ENGLISH',
 'ENGLISH',
 'ENGLISH',
 'ENGLISH',
 'ENGLISH',
 'ENGLISH',
 'ENGLISH',
 'ENGLISH',
 'ENGLISH',
 'ENGLISH',
 'ENGLISH',
 'ENGLISH',
 'ENGLISH',
 'ENGLISH',
 'ENGLISH',
 'ENGLISH',
 'ENGLISH',
 'ENGLISH',
 'FRENCH',
 'ENGLISH',
 'ENGLISH',
 'FRENCH',
 'ENGLISH',
 'ENGLISH',
 'ENGLISH',
 'ENGLISH',
 'ENGLISH',
 'FRENCH',
 'ENGLISH',
 'ENGLISH',
 'ENGLISH',
 'FRENCH',
 'ENGLISH',
 'ENGLISH',
 'ENGLISH',
 'ENGLISH',
 'ENGLISH',
 'Unknown',
 'ENGLISH',
 'ENGLISH',
 'ENGLISH',
 'ENGLISH',
 'ENGLISH',
 'ENGLISH',
 'ENGLISH',
 'ENGLISH',
 'Unknown',
 'FRENCH',
 'ENGLISH',
 'FRENCH',
 'ENGLISH',
 'ENGLISH',
 'ENGLISH',
 'ENGLISH',
 'ENGLISH',
 'ENGLISH',
 'ENGLISH',
 'ENGLISH',
 'ENGLISH',
 'ENGLISH',
 'ENGLISH',
 'ENGLISH',
 'ENGLISH',
 'ENGLISH',
 'ENGLISH',
 'ENGLISH',
 'ENGLISH',


In [19]:
df['languages'] = langs
df

,Text,new_text,languages
0,want find way end racism try force people inje...,want find way end racism try force people inje...,ENGLISH
1,"this child almost died tetanus, spent months h...","this child almost died tetanus, spent months h...",ENGLISH
2,juste se rappeler m. leblanc la pression que l...,juste se rappeler m. leblanc la pression que l...,FRENCH
3,"you would expect cases rise significantly, cou...","you would expect cases rise significantly, cou...",ENGLISH
4,depends whether vaccination status is embedded...,depends whether vaccination status is embedded...,ENGLISH
...,...,...,...
423588,"if you are eligible, please pre- register covi...","if you are eligible, please pre- register covi...",ENGLISH
423590,do it others.... 💪🎉💉🤙 #vaccinesaveslives,do it others.... flexed_bicepsparty_poppersyri...,ENGLISH
423592,#fauciouchie #2 #vaccinated #vaccinesaveslives...,#fauciouchie #2 #vaccinated #vaccinesaveslives...,Unknown
423593,yay 🙌🏻 congrats dr. j being fully vaccinated i...,yay raising_handslight_skin_tone congrats dr. ...,ENGLISH


In [20]:
df[df['languages'] != 'ENGLISH'].head()
#check how many rows

,Text,new_text,languages
2,juste se rappeler m. leblanc la pression que l...,juste se rappeler m. leblanc la pression que l...,FRENCH
13,vaccination,vaccination,Unknown
15,1-je comprends exactement ce qui doit être com...,1-je comprends exactement ce qui doit être com...,FRENCH
35,environ 500 médecins spécialistes participaien...,environ 500 médecins spécialistes participaien...,FRENCH
38,"c'est bien dommage que ce soit un droit, leur ...","c'est bien dommage que ce soit un droit, leur ...",FRENCH


In [21]:
df[df['languages'] != 'ENGLISH']['new_text']

2         juste se rappeler m. leblanc la pression que l...
13                                              vaccination
15        1-je comprends exactement ce qui doit être com...
35        environ 500 médecins spécialistes participaien...
38        c'est bien dommage que ce soit un droit, leur ...
                                ...                        
423579                             ayep. #vaccinesaveslives
423582    continue wear your mask preferably double , ge...
423585        #vaccinated #vaccinesaveslives syringemicrobe
423586    #vaccine #maskup #vaccinesaveslives #vaccinesi...
423592    #fauciouchie #2 #vaccinated #vaccinesaveslives...
Name: new_text, Length: 56002, dtype: object

In [91]:
df[df['languages'] != 'ENGLISH']['new_text'][105977]

'it’s time you do your part this fight against covid we reward you your actions we are sure we together be victorious this fight with collective effort everyone raising_handsmedium_skin_tone #vaccinesaveslives #weareinthistogether #plasmadonation #spreadpositivity'

In [22]:
df.to_csv('usa_after_language_detection.csv')

## Language Translation

In [1]:
# df = pd.read_csv('../data/usa_after_language_detection.csv')

In [ ]:
from deep_translator import GoogleTranslator

non_english = df[df['languages'] != 'ENGLISH']['new_text']

translated = []

for each in non_english:
    translate = GoogleTranslator(source='auto', target='en').translate(each)
    translated.append(translate)

In [ ]:
translated

In [ ]:
#creating new_text column
df['new_text'] = df['Text']

In [ ]:
non_english = df[df['languages'] != 'ENGLISH']

In [ ]:
non_english.head()
#33981 rows

In [ ]:
indexes_list = non_english.index

In [ ]:
indexes_list

In [ ]:
len(indexes_list)

In [ ]:
# range_firstNum = start_range + start_batch

# new_indexes_list = []

# for num in indexes_list:
#     new = num - range_firstNum
#     new_indexes_list.append(new)

In [ ]:
for i in range(0,len(indexes_list)):
    index = indexes_list[i]
    df.iloc[index]['new_text'] = translated[i]

In [ ]:
#check that the new_text col to see to check that the initial non-english rows have been translated
df[df['languages'] != 'ENGLISH']

## Language Detection Part 2

In [ ]:
# import pycld2 as cld2

# reviews = df['new_text_after_translation']
# language2 = []

# for each in reviews:
#     text_content = each
#     _, _, _, detected_language = cld2.detect(text_content, returnVectors=True)
#     language2.append(detected_language)

In [ ]:
# langs2 = []

# for each in language2:
#     lang_detected = ""
#     for i in range(0,len(each)):
#         lang_extracted = each[i][2]
#         if i != (len(each) - 1):
#             lang_detected += lang_extracted + ", "
#         else:
#             lang_detected += lang_extracted
#     langs2.append(lang_detected)

In [ ]:
# langs2

In [ ]:
# df['new_languages'] = langs2

In [ ]:
# df.to_csv("usa-after-langdetectandtrans.csv")

In [ ]:
# non_english2 = df[df['new_languages'] != 'ENGLISH']
# non_english2

In [ ]:
#see how many still detected as non-english, brieflcheck is it rly non english
# non_english2.info()

In [ ]:
# non_english2.to_csv('non_english2.csv')

## Read df after translation (USA)

In [63]:
df = pd.read_csv('./df_after_translation_usa.csv')

In [64]:
df.head()

,Unnamed: 0,Text,new_text,languages,new_text_after_translation
0,0,want find way end racism try force people inje...,want find way end racism try force people inje...,ENGLISH,want find way end racism try force people inje...
1,1,"this child almost died tetanus, spent months h...","this child almost died tetanus, spent months h...",ENGLISH,"this child almost died tetanus, spent months h..."
2,2,juste se rappeler m. leblanc la pression que l...,juste se rappeler m. leblanc la pression que l...,FRENCH,just remember mr. leblanc the pressure that th...
3,3,"you would expect cases rise significantly, cou...","you would expect cases rise significantly, cou...",ENGLISH,"you would expect cases rise significantly, cou..."
4,4,depends whether vaccination status is embedded...,depends whether vaccination status is embedded...,ENGLISH,depends whether vaccination status is embedded...


In [65]:
df

,Unnamed: 0,Text,new_text,languages,new_text_after_translation
0,0,want find way end racism try force people inje...,want find way end racism try force people inje...,ENGLISH,want find way end racism try force people inje...
1,1,"this child almost died tetanus, spent months h...","this child almost died tetanus, spent months h...",ENGLISH,"this child almost died tetanus, spent months h..."
2,2,juste se rappeler m. leblanc la pression que l...,juste se rappeler m. leblanc la pression que l...,FRENCH,just remember mr. leblanc the pressure that th...
3,3,"you would expect cases rise significantly, cou...","you would expect cases rise significantly, cou...",ENGLISH,"you would expect cases rise significantly, cou..."
4,4,depends whether vaccination status is embedded...,depends whether vaccination status is embedded...,ENGLISH,depends whether vaccination status is embedded...
...,...,...,...,...,...
404260,404260,"if you are eligible, please pre- register covi...","if you are eligible, please pre- register covi...",ENGLISH,"if you are eligible, please pre- register covi..."
404261,404261,do it others.... 💪🎉💉🤙 #vaccinesaveslives,do it others.... flexed_bicepsparty_poppersyri...,ENGLISH,do it others.... flexed_bicepsparty_poppersyri...
404262,404262,#fauciouchie #2 #vaccinated #vaccinesaveslives...,#fauciouchie #2 #vaccinated #vaccinesaveslives...,ENGLISH,#fauciouchie #2 #vaccinated #vaccinesaveslives...
404263,404263,yay 🙌🏻 congrats dr. j being fully vaccinated i...,yay raising_handslight_skin_tone congrats dr. ...,ENGLISH,yay raising_handslight_skin_tone congrats dr. ...


In [66]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [67]:
df

,Text,new_text,languages,new_text_after_translation
0,want find way end racism try force people inje...,want find way end racism try force people inje...,ENGLISH,want find way end racism try force people inje...
1,"this child almost died tetanus, spent months h...","this child almost died tetanus, spent months h...",ENGLISH,"this child almost died tetanus, spent months h..."
2,juste se rappeler m. leblanc la pression que l...,juste se rappeler m. leblanc la pression que l...,FRENCH,just remember mr. leblanc the pressure that th...
3,"you would expect cases rise significantly, cou...","you would expect cases rise significantly, cou...",ENGLISH,"you would expect cases rise significantly, cou..."
4,depends whether vaccination status is embedded...,depends whether vaccination status is embedded...,ENGLISH,depends whether vaccination status is embedded...
...,...,...,...,...
404260,"if you are eligible, please pre- register covi...","if you are eligible, please pre- register covi...",ENGLISH,"if you are eligible, please pre- register covi..."
404261,do it others.... 💪🎉💉🤙 #vaccinesaveslives,do it others.... flexed_bicepsparty_poppersyri...,ENGLISH,do it others.... flexed_bicepsparty_poppersyri...
404262,#fauciouchie #2 #vaccinated #vaccinesaveslives...,#fauciouchie #2 #vaccinated #vaccinesaveslives...,ENGLISH,#fauciouchie #2 #vaccinated #vaccinesaveslives...
404263,yay 🙌🏻 congrats dr. j being fully vaccinated i...,yay raising_handslight_skin_tone congrats dr. ...,ENGLISH,yay raising_handslight_skin_tone congrats dr. ...


## Text File & Corpus Creation

In [8]:
# create txt file for each text review

# def CreateCorpusFromDataFrame(corpusfolder, df):
#     for index, r in df.iterrows():
#     # print (r['text'])
#     # print (str(index) + '.txt')
#         text = r['new_text_after_translation']
#         fname = str(index)+'.txt'
#         corpusfile = open(corpusfolder+'/'+fname, "w", encoding="utf-8")
#         corpusfile.write(str(text))
#         corpusfile.close()

# CreateCorpusFromDataFrame('../text-corpus-usa-v2', df)

# reference: https://stackoverflow.com/questions/49088978/how-to-create-corpus-from-pandas-data-frame-to-operate-with-nltk/49104725

In [10]:
file_directory = '../text-corpus-usa-v2'
filename_pattern = '.+\.txt'

my_corpus = PlaintextCorpusReader(file_directory, filename_pattern)

#Print some words in specific file
test = my_corpus.words('0.txt')
print(test)
print(len(test))

['want', 'find', 'way', 'end', 'racism', 'try', ...]
52


In [11]:
#Total number of text files i.e. total number of tweets
print(len(my_corpus.fileids()))

404265


In [12]:
fdist = nltk.FreqDist(test)
print(fdist.most_common(10))

[('people', 4), ('.', 4), ('i', 3), ('force', 2), ('against', 2), ('vaxx', 2), ('want', 1), ('find', 1), ('way', 1), ('end', 1)]


In [ ]:
# word distribution across all files
fdist_all_files = nltk.FreqDist(my_corpus.words())
print(fdist_all_files.most_common(10))

In [ ]:
#total number of words
total_words = my_corpus.words()
print(total_words)
print(len(total_words))

## Tokenization

In [68]:
#Creating a sorted list of texts since my_corpus.fileids() does not read texts consecutively
#EDIT RANGE
sorted_list = [f'{i}.txt' for i in range(0, 2000)]

In [69]:
tokenised_words = [my_corpus.words(each) for each in sorted_list]
tokenised_words

[['want', 'find', 'way', 'end', 'racism', 'try', ...],
 ['this', 'child', 'almost', 'died', 'tetanus', ',', ...],
 ['just', 'remember', 'mr', '.', 'leblanc', 'the', ...],
 ['you', 'would', 'expect', 'cases', 'rise', ...],
 ['depends', 'whether', 'vaccination', 'status', 'is', ...],
 ['it', 'could', 'have', 'ripped', '…', 'harder', 'or', ...],
 ['they', 'do', 'this', 'at', 'vaccination', 'centres', ...],
 ['part', 'vanguard', 'vaccination', 'awareness', ...],
 ['correct', '.', 'it', '’', 's', 'not', 'about', ...],
 ['does', 'it', 'hurt', 'your', 'head', 'actually', ...],
 ['world', '-', 'wide', 'mass', 'vaccination', ...],
 ['as', 'vaccinations', '...', 'you', "'", 'll', ...],
 ['following', 'tweet', 'didn', "'", 't', 'even', 'age', ...],
 ['vaccination'],
 ['private', 'businesses', 'can', 'enforce', ...],
 ['1', '-', 'I', 'understand', 'exactly', 'what', ...],
 ['story', 'with', 'martha', 'mccallum', ',', 'am', ...],
 ['world', "'", 's', 'top', 'virologist', ',', 'luc', ...],
 ['obviou

In [70]:
print(len(tokenised_words))

2000


## Converting British to American English

In [73]:
#https://stackoverflow.com/questions/42329766/python-nlp-british-english-vs-american-english

import requests

def americanize(word):
    url ="https://raw.githubusercontent.com/hyperreality/American-British-English-Translator/master/data/british_spellings.json"
    british_to_american_dict = requests.get(url).json()

    for british_spelling, american_spelling in british_to_american_dict.items():
        word = word.replace(british_spelling, american_spelling)
  
    return word

In [74]:
test_americanize = [['flavour', 'colour'], ['apologise', 'travelled']]
test_americanize = [[americanize(w) for w in each] for each in test_americanize]
test_americanize

[['flavor', 'color'], ['apologize', 'traveled']]

In [77]:
tokenised = [[americanize(w) for w in each] for each in tokenised_words]
tokenised

In [26]:
# save americanized tokens
convert_to_str = []

for each in tokenised:
    converted_str = ""
    for i in range(0,len(each)):
        if i == (len(each) - 1):
            converted_str += each[i]
        else:
            converted_str += each[i] + ", "
    convert_to_str.append(converted_str)
    
converted_df = pd.DataFrame(convert_to_str)

converted_df.to_csv('usa_americanized_tokens_0-2000.csv')

## Conversion to numerals

In [27]:
#check if text has numbers 
def hasNumbers(inputList):
    for list in inputList:
        for word in list:
            if word.isdigit():
                return True
    return False

hasNumbers(tokenised)

True

In [28]:
# since theres numbers, the numbers need to be converted to words e.g. "2" to "two"

p = inflect.engine()
  
# convert number into words
def convert_number(text):
    # split string into list of words
    temp_str = text.split()
    # initialise empty list
    new_string = []
  
    for word in temp_str:
        # if word is a digit, convert the digit
        # to numbers and append into the new_string list
        if word.isdigit():
            temp = p.number_to_words(word)
            new_string.append(temp)
  
        # append the word as it is
        else:
            new_string.append(word)
  
    # join the words of new_string to form a string
    temp_str = ' '.join(new_string)
    return temp_str

In [29]:
# example text which contain numbers in it
input_list = ['There are 3 balls in this bag, and 12 in the other one.', '29']
result = [convert_number(each)for each in input_list]
result

['There are three balls in this bag, and twelve in the other one.',
 'twenty-nine']

In [30]:
#NEED TO CHANGE VARIABLE after us to brit eng conversion

tokenised_numerals_converted = [[convert_number(w) for w in each] for each in tokenised]

In [31]:
#check if text has numbers 
def hasNumbers(inputList):
    for list in inputList:
        for word in list:
            if word.isdigit():
                return True
    return False

hasNumbers(tokenised_numerals_converted)

False

## Misspellings

In [32]:
def correct_typo(tokens):
    spell = SpellChecker()
    return [spell.correction(t) if len(spell.unknown([t]))>0 else t for t in tokens]

In [33]:
misspelled_test = ['greot', 'coyor', 'and', 'misspeling']
correct_typo(misspelled_test)

['great', 'color', 'and', 'misspelling']

In [34]:
tokenised_spellchecked = [correct_typo(each) for each in tokenised_numerals_converted]

In [35]:
tokenised_spellchecked

[['want',
  'find',
  'way',
  'end',
  'racism',
  'try',
  'force',
  'people',
  'inject',
  'themselves',
  '.',
  'that',
  'i',
  'i',
  'what',
  'i',
  'see',
  'bunch',
  'people',
  'all',
  'races',
  'standing',
  'up',
  'against',
  'forced',
  'vaccination',
  '.',
  'not',
  'against',
  'maxx',
  '.',
  'some',
  'people',
  'need',
  'these',
  'maxx',
  '&',
  'amp',
  ';',
  'i',
  'support',
  'those',
  'people',
  'getting',
  'their',
  'shot',
  '.',
  'i',
  'draw',
  'line',
  'at',
  'force'],
 ['this',
  'child',
  'almost',
  'died',
  'tetanus',
  ',',
  'spent',
  'months',
  'hospital',
  'at',
  'cost',
  'around',
  '$',
  'am',
  ',',
  'his',
  'parents',
  'still',
  'refused',
  'vaccination',
  'after',
  '.',
  'this',
  'was',
  'pre',
  '-',
  'pandemic',
  '.'],
 ['just',
  'remember',
  'my',
  '.',
  'leblanc',
  'the',
  'pressure',
  'that',
  'the',
  'provincial',
  'am',
  'and',
  'the',
  'business',
  'world',
  'asked',
  'for',
  

In [36]:
convert_to_str = []

for each in tokenised_spellchecked:
    converted_str = ""
    for i in range(0,len(each)):
        if i == (len(each) - 1):
            converted_str += each[i]
        else:
            converted_str += each[i] + ", "
    convert_to_str.append(converted_str)
    
converted_df = pd.DataFrame(convert_to_str)

converted_df.to_csv('corrected-0-2000.csv')

In [37]:
converted_df

,0
0,"want, find, way, end, racism, try, force, peop..."
1,"this, child, almost, died, tetanus, ,, spent, ..."
2,"just, remember, my, ., leblanc, the, pressure,..."
3,"you, would, expect, cases, rise, significantly..."
4,"depends, whether, vaccination, status, is, emb..."
...,...
1995,"this, logic, is, valid, today, but, not, one, ..."
1996,"we, need, mandate, couid, vaccination, i, -, t..."
1997,"do, you, mean, vaccination, because, you, can,..."
1998,"be, fair, i, have, also, visited, more, restau..."


## Compiled CSV

In [ ]:
df2 = pd.read_csv("compiled-after-correction.csv")

In [ ]:
new_tokenised_spellchecked = []

for row in df2['0']:
    alist = row.split(", ")
    new_tokenised_spellchecked.append(alist)

In [ ]:
new_tokenised_spellchecked

## Convert to lowercase

In [50]:
#NEED TO CHANGE VARIABLE AFTER SPELLCHECK

tokenised_lower = [[w.lower() for w in each] for each in tokenised_numerals_converted]
tokenised_lower

[['want',
  'find',
  'way',
  'end',
  'racism',
  'try',
  'force',
  'people',
  'inject',
  'themselves',
  '.',
  'that',
  '’',
  's',
  'what',
  'i',
  'see',
  'bunch',
  'people',
  'all',
  'races',
  'standing',
  'up',
  'against',
  'forced',
  'vaccination',
  '.',
  'not',
  'against',
  'vaxx',
  '.',
  'some',
  'people',
  'need',
  'these',
  'vaxx',
  '&',
  'amp',
  ';',
  'i',
  'support',
  'those',
  'people',
  'getting',
  'their',
  'shot',
  '.',
  'i',
  'draw',
  'line',
  'at',
  'force'],
 ['this',
  'child',
  'almost',
  'died',
  'tetanus',
  ',',
  'spent',
  'months',
  'hospital',
  'at',
  'cost',
  'around',
  '$',
  '1m',
  ',',
  'his',
  'parents',
  'still',
  'refused',
  'vaccination',
  'after',
  '.',
  'this',
  'was',
  'pre',
  '-',
  'pandemic',
  '.'],
 ['just',
  'remember',
  'mr',
  '.',
  'leblanc',
  'the',
  'pressure',
  'that',
  'the',
  'provincial',
  'pm',
  'and',
  'the',
  'business',
  'world',
  'asked',
  'for',
  

## Remove punctuation

In [51]:
# remove punctuations from tokenised_lower
tokenised_words_only = [[w for w in each if re.search('^[a-z]+$',w)] for each in tokenised_lower]
tokenised_words_only

[['want',
  'find',
  'way',
  'end',
  'racism',
  'try',
  'force',
  'people',
  'inject',
  'themselves',
  'that',
  's',
  'what',
  'i',
  'see',
  'bunch',
  'people',
  'all',
  'races',
  'standing',
  'up',
  'against',
  'forced',
  'vaccination',
  'not',
  'against',
  'vaxx',
  'some',
  'people',
  'need',
  'these',
  'vaxx',
  'amp',
  'i',
  'support',
  'those',
  'people',
  'getting',
  'their',
  'shot',
  'i',
  'draw',
  'line',
  'at',
  'force'],
 ['this',
  'child',
  'almost',
  'died',
  'tetanus',
  'spent',
  'months',
  'hospital',
  'at',
  'cost',
  'around',
  'his',
  'parents',
  'still',
  'refused',
  'vaccination',
  'after',
  'this',
  'was',
  'pre',
  'pandemic'],
 ['just',
  'remember',
  'mr',
  'leblanc',
  'the',
  'pressure',
  'that',
  'the',
  'provincial',
  'pm',
  'and',
  'the',
  'business',
  'world',
  'asked',
  'for',
  'it',
  'to',
  'be',
  'able',
  'to',
  're',
  'open',
  'the',
  'borders',
  'air',
  'traffic',
  'w

## Remove words len <= 3

In [52]:
new_list = []

for alist in tokenised_words_only:
    mini_list = []
    for word in alist:
        if len(word) > 3:
            mini_list.append(word)
    new_list.append(mini_list)

In [53]:
#test
new_list

[['want',
  'find',
  'racism',
  'force',
  'people',
  'inject',
  'themselves',
  'that',
  'what',
  'bunch',
  'people',
  'races',
  'standing',
  'against',
  'forced',
  'vaccination',
  'against',
  'vaxx',
  'some',
  'people',
  'need',
  'these',
  'vaxx',
  'support',
  'those',
  'people',
  'getting',
  'their',
  'shot',
  'draw',
  'line',
  'force'],
 ['this',
  'child',
  'almost',
  'died',
  'tetanus',
  'spent',
  'months',
  'hospital',
  'cost',
  'around',
  'parents',
  'still',
  'refused',
  'vaccination',
  'after',
  'this',
  'pandemic'],
 ['just',
  'remember',
  'leblanc',
  'pressure',
  'that',
  'provincial',
  'business',
  'world',
  'asked',
  'able',
  'open',
  'borders',
  'traffic',
  'will',
  'provincial',
  'govt',
  'included',
  'block',
  'tourists',
  'will',
  'have',
  'pass',
  'vaccination',
  'hotel',
  'restaurant'],
 ['would',
  'expect',
  'cases',
  'rise',
  'significantly',
  'course',
  'vaccination',
  'rates',
  'come',
  

In [54]:
tokenised_words_only = new_list

In [55]:
tokenised_words_only

[['want',
  'find',
  'racism',
  'force',
  'people',
  'inject',
  'themselves',
  'that',
  'what',
  'bunch',
  'people',
  'races',
  'standing',
  'against',
  'forced',
  'vaccination',
  'against',
  'vaxx',
  'some',
  'people',
  'need',
  'these',
  'vaxx',
  'support',
  'those',
  'people',
  'getting',
  'their',
  'shot',
  'draw',
  'line',
  'force'],
 ['this',
  'child',
  'almost',
  'died',
  'tetanus',
  'spent',
  'months',
  'hospital',
  'cost',
  'around',
  'parents',
  'still',
  'refused',
  'vaccination',
  'after',
  'this',
  'pandemic'],
 ['just',
  'remember',
  'leblanc',
  'pressure',
  'that',
  'provincial',
  'business',
  'world',
  'asked',
  'able',
  'open',
  'borders',
  'traffic',
  'will',
  'provincial',
  'govt',
  'included',
  'block',
  'tourists',
  'will',
  'have',
  'pass',
  'vaccination',
  'hotel',
  'restaurant'],
 ['would',
  'expect',
  'cases',
  'rise',
  'significantly',
  'course',
  'vaccination',
  'rates',
  'come',
  

## POS Tagging

In [56]:
tokens = tokenised_words_only[0]
tokens

['want',
 'find',
 'racism',
 'force',
 'people',
 'inject',
 'themselves',
 'that',
 'what',
 'bunch',
 'people',
 'races',
 'standing',
 'against',
 'forced',
 'vaccination',
 'against',
 'vaxx',
 'some',
 'people',
 'need',
 'these',
 'vaxx',
 'support',
 'those',
 'people',
 'getting',
 'their',
 'shot',
 'draw',
 'line',
 'force']

In [57]:
nltk.pos_tag(tokens)

[('want', 'JJ'),
 ('find', 'VBP'),
 ('racism', 'NN'),
 ('force', 'NN'),
 ('people', 'NNS'),
 ('inject', 'VBP'),
 ('themselves', 'PRP'),
 ('that', 'IN'),
 ('what', 'WP'),
 ('bunch', 'JJ'),
 ('people', 'NNS'),
 ('races', 'NNS'),
 ('standing', 'VBG'),
 ('against', 'IN'),
 ('forced', 'VBN'),
 ('vaccination', 'NN'),
 ('against', 'IN'),
 ('vaxx', 'FW'),
 ('some', 'DT'),
 ('people', 'NNS'),
 ('need', 'VBP'),
 ('these', 'DT'),
 ('vaxx', 'JJ'),
 ('support', 'NN'),
 ('those', 'DT'),
 ('people', 'NNS'),
 ('getting', 'VBG'),
 ('their', 'PRP$'),
 ('shot', 'JJ'),
 ('draw', 'JJ'),
 ('line', 'NN'),
 ('force', 'NN')]

In [58]:
pos_tagged = []

for each in tokenised_words_only:
    pos_tags = nltk.pos_tag(each)
    pos_tagged.append(pos_tags)

In [59]:
pos_tagged

[[('want', 'JJ'),
  ('find', 'VBP'),
  ('racism', 'NN'),
  ('force', 'NN'),
  ('people', 'NNS'),
  ('inject', 'VBP'),
  ('themselves', 'PRP'),
  ('that', 'IN'),
  ('what', 'WP'),
  ('bunch', 'JJ'),
  ('people', 'NNS'),
  ('races', 'NNS'),
  ('standing', 'VBG'),
  ('against', 'IN'),
  ('forced', 'VBN'),
  ('vaccination', 'NN'),
  ('against', 'IN'),
  ('vaxx', 'FW'),
  ('some', 'DT'),
  ('people', 'NNS'),
  ('need', 'VBP'),
  ('these', 'DT'),
  ('vaxx', 'JJ'),
  ('support', 'NN'),
  ('those', 'DT'),
  ('people', 'NNS'),
  ('getting', 'VBG'),
  ('their', 'PRP$'),
  ('shot', 'JJ'),
  ('draw', 'JJ'),
  ('line', 'NN'),
  ('force', 'NN')],
 [('this', 'DT'),
  ('child', 'NN'),
  ('almost', 'RB'),
  ('died', 'VBD'),
  ('tetanus', 'JJ'),
  ('spent', 'JJ'),
  ('months', 'NNS'),
  ('hospital', 'JJ'),
  ('cost', 'NN'),
  ('around', 'IN'),
  ('parents', 'NNS'),
  ('still', 'RB'),
  ('refused', 'VBD'),
  ('vaccination', 'NN'),
  ('after', 'IN'),
  ('this', 'DT'),
  ('pandemic', 'JJ')],
 [('just', 'RB

In [76]:
df['pos_tagged_words'] = pos_tagged

ValueError: Length of values (2000) does not match length of index (404265)

In [ ]:
df.head()

## Stop word removal

In [ ]:
# TEST: remove stop words from test_words_only
from nltk.corpus import stopwords
stop_list = stopwords.words('english')

test_words_only1 = ['not', 'great', 'color', 'and', 'cruelty', 'free', 'i', 'use', 'elf', 'cosmetics', 'every', 'day', 'and', 'i', 'love', 'it']
test_stopremoved = [w for w in test_words_only1 if w not in stop_list or w == 'not']
print(test_stopremoved)

In [ ]:
# remove stop words from tokenised_words_only
from nltk.corpus import stopwords
stop_list = stopwords.words('english')
tokenised_stopremoved = [[w for w in each if w not in stop_list or w == 'not'] for each in tokenised_words_only]
tokenised_stopremoved

## Bigram & Trigram Models

In [ ]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(tokenised_stopremoved, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[tokenised_stopremoved], threshold=100)
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [ ]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]
def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

In [ ]:
# Form Bigrams
tokenised_bigrams = make_bigrams(tokenised_stopremoved)
# Form Trigrams
tokenised_trigrams = make_trigrams(tokenised_stopremoved)

In [ ]:
tokenised_bigrams

In [ ]:
tokenised_trigrams

## Lemmatization

### Lemmatization with all postags

In [ ]:
# TEST: lemmatisation of test_stopremoved
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()
test_lemmatized = [lemmatizer.lemmatize(w)for w in test_stopremoved]
print(test_lemmatized)

In [ ]:
#normal lemmatization of tokenised_stopremoved
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()
tokenised_lemmatized_1 = [[lemmatizer.lemmatize(w) for w in each] for each in tokenised_bigrams]
tokenised_lemmatized_1

### Lemmatisation with noun, adj, verb and adv

In [ ]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
import spacy 

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
tokenised_lemmatized_2 = lemmatization(tokenised_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
tokenised_lemmatized_2

### Lemmatisation with nouns only

In [ ]:
def lemmatization_noun(texts, allowed_postags=['NOUN']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
import spacy 

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
tokenised_lemmatized_3 = lemmatization_noun(tokenised_bigrams, allowed_postags=['NOUN'])
tokenised_lemmatized_3

### Gibberish Remover

In [ ]:
#test
trial_nested=[['happy', 'time', 'ajsdnkfjak'],[],[],['dsnfkajka','yo','wassup']]

In [ ]:
#test
len([])

if len([])== False:
    print('hi')

In [ ]:
#test
from gibberish_detector import detector
# load the gibberish detection model
Detector = detector.create_from_model('../Gibberish-Detector-master/gibberish-detector.model')

##change variable tokenised_lemmatized_x

new_list = []

for alist in trial_nested:
    mini_list = []
    if len(alist):
        for word in alist:
            test = Detector.is_gibberish(word)
            if test == False:
                mini_list.append(word)
    new_list.append(mini_list)

In [ ]:
#test
new_list

In [ ]:
#this is for tokenised_lemmatized_1
from gibberish_detector import detector
# load the gibberish detection model
Detector = detector.create_from_model('../Gibberish-Detector-master/gibberish-detector.model')

##change variable tokenised_lemmatized_x
new_list = []

for alist in tokenised_lemmatized_1:
    mini_list = []
    if len(alist):
        for word in alist:
            test = Detector.is_gibberish(word)
            if test == False:
                mini_list.append(word)
    new_list.append(mini_list)
    
tokenised_lemmatized_1 = new_list

In [ ]:
tokenised_lemmatized_1

In [ ]:
#this is for tokenised_lemmatized_2
from gibberish_detector import detector
# load the gibberish detection model
Detector = detector.create_from_model('../Gibberish-Detector-master/gibberish-detector.model')

##change variable tokenised_lemmatized_x
new_list2 = []

for alist in tokenised_lemmatized_2:
    mini_list = []
    if len(alist):
        for word in alist:
            test = Detector.is_gibberish(word)
            if test == False:
                mini_list.append(word)
    new_list2.append(mini_list)
                
tokenised_lemmatized_2 = new_list2

In [ ]:
tokenised_lemmatized_2

In [ ]:
#this is for tokenised_lemmatized_3
from gibberish_detector import detector
# load the gibberish detection model
Detector = detector.create_from_model('../Gibberish-Detector-master/gibberish-detector.model')

##change variable tokenised_lemmatized_x
new_list3 = []

for alist in tokenised_lemmatized_3:
    mini_list = []
    if len(alist):
        for word in alist:
            test = Detector.is_gibberish(word)
            if test == False:
                mini_list.append(word)
    new_list3.append(mini_list)
                
tokenised_lemmatized_3 = new_list3

In [ ]:
tokenised_lemmatized_3

## Creating column with Pre-processed words

In [ ]:
df["words_processed_all"] = tokenised_lemmatized_1
df["words_processed_noun_adj_verb_adv"] = tokenised_lemmatized_2
df["words_processed_noun"] = tokenised_lemmatized_3

In [ ]:
df.head()

In [ ]:
df.to_csv('usa-v2-preprocessed.csv')

## TFIDF

In [ ]:
df['words_processed_all']

In [ ]:
def listToString(s): 
    # initialize an empty string
    str1 = "" 
    # traverse in the string  
    for ele in s: 
        str1 += ele  
    # return string  
    return str1

In [ ]:
newlist = []

for each in df['words_processed_all']:
    new = each.strip("][").replace("'","").split(", ")
    newlist.append(new)

In [ ]:
newlist

In [ ]:
joined_list = []

for lis in newlist:
    new = " ".join(lis)
    joined_list.append(new)

In [ ]:
joined_list

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
import pandas as pd
cv = CountVectorizer()
word_count_vector = cv.fit_transform(joined_list)
tf = pd.DataFrame(word_count_vector.toarray(), columns=cv.get_feature_names())
print(tf)

In [ ]:
tfidf_transformer = TfidfTransformer()
X = tfidf_transformer.fit_transform(word_count_vector)
idf = pd.DataFrame({'feature_name':cv.get_feature_names(), 'idf_weights':tfidf_transformer.idf_})
idf.sort_values('idf_weights', ascending=False, inplace=True)

In [ ]:
idf.to_csv('tfidf-descending.csv')

In [ ]:
tf_idf = pd.DataFrame(X.toarray(),columns=cv.get_feature_names())
tf_idf